# 前置

In [1]:
import pandas as pd
from openpyxl import Workbook
from openpyxl.utils.dataframe import dataframe_to_rows
import numpy as np

In [2]:
# 读取日报数据
file_path = r'D:\Ori\日报.xlsx'
sheet_name = '日报by渠道'
df = pd.read_excel(file_path, sheet_name=sheet_name)
# 读取第二个Sheet
sheet_name2 = '日报by优惠券'
df2 = pd.read_excel(file_path, sheet_name=sheet_name2)

# 时期定义

In [3]:
# 定义一个函数来计算每周的开始和结束日期
def get_custom_week_period(date):
    # 自定义的周期起始日期
    start_date = pd.Timestamp('2025-01-02')
    
    # 计算当前日期距离起始日期的天数
    delta_days = (date - start_date).days
    
    # 计算当前日期属于第几个周期
    week_number = delta_days // 7
    
    # 计算周期的开始和结束日期
    period_start = start_date + pd.Timedelta(days=week_number * 7)
    period_end = period_start + pd.Timedelta(days=6)
    
    return f"{period_start.strftime('%Y-%m-%d')}至{period_end.strftime('%Y-%m-%d')}"

In [4]:
#### 新增一列时期，代表每周的周期
df['时期'] = df['日期'].apply(get_custom_week_period)

# 第一个sheet

In [5]:
# 按周周期进行groupby汇总
weekly_report = df.groupby(['时期','城市']).agg({
    '触达用户数': 'sum',
    '点击用户数': 'sum',
    '领券用户数': 'sum',
    '交易用户数': 'sum',
    '交易订单量': 'sum',
    '核销用户数': 'sum',
    '交易新用户数': 'sum',
    '券补金额': 'sum',
    '渠道成本': 'sum'
}).reset_index()

In [6]:
weekly_report['未核销用户数'] = weekly_report['交易用户数'] - weekly_report['核销用户数']
weekly_report['触达点击率'] = weekly_report['点击用户数'] / weekly_report['触达用户数']
weekly_report['点击转化率'] = weekly_report['交易用户数'] / weekly_report['点击用户数']
weekly_report['支付核销率'] = weekly_report['核销用户数'] / weekly_report['交易用户数']
weekly_report['日均交易订单量'] = weekly_report['交易订单量'] / 7
weekly_report['团单CAC'] = (weekly_report['券补金额'] + weekly_report['渠道成本']) / weekly_report['交易用户数']

# 转换前三个率为百分比格式
rate_columns = ['触达点击率', '点击转化率', '支付核销率']
weekly_report[rate_columns] = weekly_report[rate_columns].apply(lambda x: (x * 100).round(2).astype(str) + '%')

# 保留后两列为两位小数
decimal_columns = ['日均交易订单量', '团单CAC']
weekly_report[decimal_columns] = weekly_report[decimal_columns].round(2)

In [7]:
# 创建一个新的Excel工作簿
wb = Workbook()

ws1 = wb.active
ws1.title = '线上获客概览'

# 写入数据到新的Sheet
for r in dataframe_to_rows(weekly_report, index=False, header=True):
    ws1.append(r)

# 第二个sheet

In [8]:
# 计算渠道获客CAC核算
cac_report = df.groupby(['时期', '城市','渠道']).agg({
    '渠道成本': 'sum',
    '券补金额': 'sum',
    '交易用户数': 'sum',
    '核销用户数': 'sum'
}).reset_index()
cac_report['总花费'] = cac_report['渠道成本'] + cac_report['券补金额']
cac_report['团单CAC'] = cac_report['总花费'] / cac_report['交易用户数']
cac_report['核销CAC'] = cac_report['总花费'] / cac_report['核销用户数']

In [9]:
# 保留两位小数
decimal_columns2 = ['总花费', '团单CAC', '核销CAC']
cac_report[decimal_columns2] = cac_report[decimal_columns2].round(2)

In [ ]:
# 创建第二个Sheet用于保存渠道获客CAC核算数据
ws2 = wb.create_sheet(title='CAC')
for r in dataframe_to_rows(cac_report, index=False, header=True):
    ws2.append(r)

# 第三个sheet

In [10]:
# 按周周期和渠道进行groupby汇总，用于渠道获客转化by周
conversion_report = df.groupby(['时期','城市', '渠道']).agg({
    '触达用户数': 'sum',
    '点击用户数': 'sum',
    '领券用户数': 'sum',
    '交易用户数': 'sum',
    '交易订单量': 'sum',
    '核销用户数': 'sum',
    '交易新用户数': 'sum',
    '券补金额': 'sum',
    '渠道成本': 'sum'
}).reset_index()

conversion_report['触达点击率'] = conversion_report['点击用户数'] / conversion_report['触达用户数']
conversion_report['点击转化率'] = conversion_report['交易用户数'] / conversion_report['点击用户数']
conversion_report['新用户占比'] = conversion_report['交易新用户数'] / conversion_report['交易用户数']

In [11]:
# 转换前三个率为百分比格式
rate_columns2 = ['触达点击率', '点击转化率', '新用户占比']
conversion_report[rate_columns2] = conversion_report[rate_columns2].apply(lambda x: (x * 100).round(2).astype(str) + '%')

# 创建第三个Sheet用于保存渠道获客转化by周数据
ws3 = wb.create_sheet(title='线上获客细则')
for r in dataframe_to_rows(conversion_report, index=False, header=True):
    ws3.append(r)

# 第四个sheet

In [12]:
df2['日期'] = pd.to_datetime(df2['日期'])
df2['时期'] = df2['日期'].apply(get_custom_week_period)
df2['是否门店新用户'] = df2['是否新客']
# 按时期、优惠券ID、面额、渠道和人群进行groupby汇总，用于人群转化by周
crowd_conversion_report = df2.groupby(['时期', '人群','是否新客','优惠券ID', '面额', '渠道','人群ID','城市']).agg({
    '领券用户数': 'sum',
    '交易用户数': 'sum',
    '交易订单量': 'sum',
    '交易新用户数': 'sum',
    '核销用户数': 'sum',
    '券补金额': 'sum'
}).reset_index()

In [13]:
# 创建第四个Sheet用于保存人群转化by周数据
ws4 = wb.create_sheet(title='人群漏斗')
for r in dataframe_to_rows(crowd_conversion_report, index=False, header=True):
    ws4.append(r)

# 第五个sheet

In [14]:
# 按周周期和渠道进行groupby汇总，用于新老客分渠道
channel_user = df2.groupby(['时期','城市', '是否新客', '渠道']).agg({
    '领券用户数': 'sum',
    '交易用户数': 'sum',
    '核销用户数': 'sum',
    '交易订单量': 'sum',
    '交易新用户数': 'sum'
}).reset_index()

In [15]:
# 创建第五个Sheet用于保存人群转化by周数据
ws5 = wb.create_sheet(title='渠道分新老')
for r in dataframe_to_rows(channel_user, index=False, header=True):
    ws5.append(r)

# 导出

In [16]:
# 保存新的Excel文件
output_file_path = r'C:\Users\1\Desktop\周报.xlsx'
wb.save(output_file_path)